In [ ]:
%load_ext autoreload
%autoreload 2

import os
import shutil
import pickle
import time
import pprint
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
from IPython.display import display, Image, SVG

from pydrake.examples.quadrotor import QuadrotorGeometry
from pydrake.geometry import MeshcatVisualizerCpp, Rgba, StartMeshcat
from pydrake.geometry.optimization import HPolyhedron, VPolytope
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.multibody.parsing import Parser
from pydrake.solvers.gurobi import GurobiSolver
from pydrake.solvers.mosek import MosekSolver
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder, LeafSystem

from models.building_generation import *
from spp.bezier import BezierSPP
from spp.rounding import *

g_lic = GurobiSolver.AcquireLicense()
m_lic = MosekSolver.AcquireLicense()

In [ ]:
def build_bezier_gcs(regions):
    order = 7
    continuity = 4
    vel_limit = 10 * np.ones(3)
    hdot_min = 1e-3
    weights = {"time": 1., "norm": 1.}
    
    gcs = BezierSPP(regions, order, continuity, hdot_min=hdot_min)
    gcs.addTimeCost(weights["time"])
    gcs.addPathLengthCost(weights["norm"])
    gcs.addVelocityLimits(-vel_limit, vel_limit)
    gcs.setPaperSolverOptions()
    gcs.setSolver(MosekSolver())
#     gcs.setSolver(GurobiSolver())
#     regularization = 1e-3
#     for derivative in [2, 3, 4]:
#         gcs.addDerivativeRegularization(regularization, regularization, derivative)
#     gcs.setRoundingStrategy(randomForwardPathSearch)

    return gcs

In [ ]:
start = np.array([-1, -1])
goal = np.array([2, 1])
start_pose = np.r_[(start-start)*5, 1.]
goal_pose = np.r_[(goal-start)*5., 1.]

zero_deriv_boundary = 3

region_file = "room_debug/regions.reg"
with open(region_file, "rb") as f:
    regions = pickle.load(f)
    
np.random.seed(0)
relax_gcs = build_bezier_gcs(regions)
relax_output = relax_gcs.SolvePath(start_pose, goal_pose, True, False,
                                   zero_deriv_boundary=zero_deriv_boundary, preprocessing=True)
b_traj_relax, result_relax, best_result_relax, hard_result_relax, _ = relax_output
    
mip_gcs = build_bezier_gcs(regions)
mip_output = mip_gcs.SolvePath(start_pose, goal_pose, False, False,
                               zero_deriv_boundary=zero_deriv_boundary)
b_traj_mip, result_mip = mip_output[:2]

In [ ]:
display(Image(relax_gcs.VisualizeGraph("png")))

In [ ]:
print(result_relax.get_optimal_cost(), best_result_relax.get_optimal_cost(), result_mip.get_optimal_cost())